# Code to generate Frustration fouls and Euro Fouls

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from nba_api.stats.static import teams
from pbpstats.client import Client
import plotly.graph_objects as go 
from tqdm import tqdm
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "./figs/"

In [9]:
settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "C:/Users/pansr/Documents/Sra_Coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
league = "nba"
season_yr = "2022"
season_type = "Regular Season"
season = client.Season(league, season_yr, season_type)
games_id = []

for final_game in season.games.final_games:
    games_id.append(final_game['game_id'])
print(len(games_id))

1230


In [10]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "C:/Users/pansr/Documents/Sra_Coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
for gameid in tqdm(games_id):
    try:
        games_list.append(client.Game(gameid))
    except:
        # print(gameid)
        bad_games_list.append(gameid)
        continue
print(len(bad_games_list))

100%|██████████| 1230/1230 [00:41<00:00, 29.40it/s]

35


In [23]:
i = 0

In [26]:
gameid

'0022200519'

In [25]:
i = 34
gameid = bad_games_list[i]
client.Game(gameid)

InvalidNumberOfStartersException: GameId: 0022200519, Period: 5, TeamId: 1610612741, Players: [202696, 1630172, 201942, 1630245]

In [4]:
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
pos_store = []
TO_Miss_pID = []
Foul_pID = []
Foul_tID = []
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= 5:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                # print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                # print ("Foul Player: {0}".format(possession_event.player1_id))

100%|██████████| 1195/1195 [00:00<00:00, 3272.76it/s]


In [7]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

In [8]:
Player_Name = []
a = Foul_pID
Foul_pID1, Fouls = np.unique(a, return_counts=True)  

for pID in Foul_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [ ]:

for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(Foul_pID1))

In [ ]:
data_euroP = pd.DataFrame({'Foul_pID':Foul_pID1,'Player_Name':Player_Name,'Fouls':Fouls})
data_euroP = data_euroP[data_euroP['Player_Name'] != 'abc']
# data_euroP.to_csv ('Fast_Fouls_20_21_updated.csv',index=False)

In [ ]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

In [ ]:
np.sum(Fouls)

In [ ]:
data_euroT = pd.DataFrame({'Foul_tID':Foul_tID1,'Team_Name':Team_Name,'Fouls':Fouls})
# data_euroT.to_csv ('Fast_Fouls_20_21_teams_updated.csv',index=False)
data_euroT

In [ ]:
FFoul_pID =[]
k = 0
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])
        k +=1
        print(k)
        clear_output(wait=True)

In [ ]:
Player_Name = []
a = FFoul_pID
FFoul_pID, Fouls = np.unique(a, return_counts=True)
for pID in FFoul_pID:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [ ]:
for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(FFoul_pID))

In [ ]:
data_frust = pd.DataFrame({'FFoul_pID':FFoul_pID,'Player':Player_Name,'Fouls':Fouls})
data_frust = data_frust[data_frust['Player'] != 'abc']
# data_frust.to_csv ('Frustration_Fouls_20_21_updated.csv',index=False)

In [ ]:
data_frust['rank'] = data_frust['Fouls'].rank(ascending=False) 
data_frust = data_frust.sort_values(by=['rank'])
data_frust_plot = data_frust.drop(columns=['FFoul_pID', 'rank'])
data_frust_plot.reset_index(drop=True,inplace=True)
data_frust_plot.reset_index(drop=False,inplace=True)
data_frust_plot["#"] = data_frust_plot["index"] +1
data_frust_plot = data_frust_plot[["#","Player","Fouls"]]
data_frust_plot = data_frust_plot.head(10)
data_frust_plot

In [ ]:
def plot_table(df,var,sort="Player",title=" ",col_width=15):
    fig = go.Figure(data=[go.Table(
        columnwidth=[5,40,col_width],
        header=dict(values=list(df.columns),
                    fill_color='blue',
                    align=['center','left','center'],
                    font=dict(color='snow',family="Arial Black, monospace", size=12),
                    line_color="snow"
                    ),
        cells=dict(values=[df["#"],df[sort],df[var]],
                fill_color='lavender',
                align=['center','left','center'],
                height=23,
                line_color="grey",#lightgrey",
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
    fig.update_layout(title=dict(text=title,y=0.98,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
    tab_width = 250 + col_width
    tab_height = 310
    fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
    fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
    fig.add_annotation(x=1.0, y=0.0,text="Source: pbpstats",showarrow=False,xshift=1,yshift=5)
    # fig.update_layout(autosize=True)
    fig.show()
    # fig.write_image(fig_DIR+f"{var}.png", scale=2)
    return fig

In [ ]:
fig = plot_table(data_frust_plot,"Fouls",title="Frustration Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)

In [ ]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

data_euro = pd.DataFrame({'Foul_tID':Foul_tID1,'Team':Team_Name,'Fouls':Fouls})
data_euro['rank'] = data_euro['Fouls'].rank(ascending=False) 
data_euro = data_euro.sort_values(by=['rank'])
data_euro_plot = data_euro.drop(columns=['Foul_tID', 'rank'])
data_euro_plot.reset_index(drop=True,inplace=True)
data_euro_plot.reset_index(drop=False,inplace=True)
data_euro_plot["#"] = data_euro_plot["index"] +1
data_euro_plot = data_euro_plot[["#","Team","Fouls"]]
data_euro_plot = data_euro_plot.head(10)
data_euro_plot

In [ ]:
fig = plot_table(data_euro_plot,"Fouls",sort="Team",title="Euro Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)